In [ ]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd

# Cargar el dataset
df = pd.read_csv('dataset.csv', parse_dates=['trans_date_trans_time'])

# Crear las columnas necesarias una vez para evitar modificaciones dentro de los callbacks
df['date'] = df['trans_date_trans_time'].dt.date
df['hour'] = df['trans_date_trans_time'].dt.hour
df['day_of_week'] = df['trans_date_trans_time'].dt.day_name()  # Día de la semana

# Crear etiquetas de franjas horarias
hour_labels = [f"{str(i).zfill(2)}:00 - {str(i+1).zfill(2)}:00" for i in range(24)]

# Obtener la lista de estados únicos para el filtro
states = df['state'].unique()
states = ['Todos'] + list(states)  # Añadir una opción "Todos" para mostrar todos los estados

# Inicializar la aplicación
app = Dash(__name__)

# Crear el layout de la aplicación
app.layout = html.Div(style={'backgroundColor': '#1f2630', 'color': 'white', 'padding': '10px'}, children=[
    html.H1('Análisis de Transacciones', style={'textAlign': 'center', 'color': '#7FDBFF'}),

    # Filtro por estado
    html.Div([
        html.Label('Selecciona el estado:', style={'color': 'white'}),
        dcc.Dropdown(
            id='state-filter',
            options=[{'label': state, 'value': state} for state in states],
            value='Todos',  # Valor por defecto
            style={'width': '50%', 'color': 'black', 'backgroundColor': 'white'}  # Texto negro, fondo blanco
        ),
    ], style={'padding': '10px'}),

    # Disposición de los gráficos en filas y columnas con estilo
    html.Div([
        html.Div([
            html.H3('Mapa de Calor de Transacciones', style={'color': '#7FDBFF'}),
            dcc.Graph(id='heatmap', style={'height': '500px'})  # Ocupa las 2 columnas
        ], style={'width': '100%', 'padding': '10px'}),  # Ajustado al 100% de ancho

        html.Div([
            html.Div([
                html.H3('Distribución de Género', style={'color': '#7FDBFF'}),
                dcc.Graph(id='gender-distribution', style={'height': '400px'})
            ], style={'width': '48%', 'display': 'inline-block', 'padding': '10px', 'verticalAlign': 'top'}),

            html.Div([
                html.H3('Distribución de Categorías de Transacción', style={'color': '#7FDBFF'}),
                dcc.Graph(id='category-distribution', style={'height': '400px'})
            ], style={'width': '48%', 'display': 'inline-block', 'padding': '10px', 'verticalAlign': 'top'}),
        ]),

        html.Div([
            html.Div([
                html.H3('Interacción por Franja Horaria y Día de la Semana', style={'color': '#7FDBFF'}),
                dcc.Graph(id='weekly-hourly-frequency', style={'height': '800px', 'width': '1200px'})  # Ajustado para mayor claridad
            ], style={'width': '100%', 'padding': '10px', 'verticalAlign': 'top'}),
        ]),
    ])
])

# Definir los callbacks para actualizar cada gráfico con estilo oscuro y el filtro de estado
@app.callback(
    Output('heatmap', 'figure'),
    Input('heatmap', 'id'),
    Input('state-filter', 'value')
)
def update_heatmap(_, selected_state):
    # Filtrar por estado
    filtered_df = df if selected_state == 'Todos' else df[df['state'] == selected_state]
    fig = px.density_mapbox(
        filtered_df, lat='lat', lon='long', z='amt', radius=10,
        center=dict(lat=37.0902, lon=-95.7129),  # Centrado en Estados Unidos
        zoom=4,
        mapbox_style="carto-darkmatter"
    )
    fig.update_layout(title="Mapa de Calor de Transacciones", paper_bgcolor='#1f2630', font_color='white')
    return fig

@app.callback(
    Output('gender-distribution', 'figure'),
    Input('state-filter', 'value')
)
def update_gender_distribution(selected_state):
    filtered_df = df if selected_state == 'Todos' else df[df['state'] == selected_state]
    fig = px.histogram(filtered_df, x='gender', color='gender', color_discrete_sequence=px.colors.sequential.Plasma)
    fig.update_layout(title="Distribución de Género", paper_bgcolor='#1f2630', plot_bgcolor='#1f2630', font_color='white')
    return fig

@app.callback(
    Output('category-distribution', 'figure'),
    Input('state-filter', 'value')
)
def update_category_distribution(selected_state):
    filtered_df = df if selected_state == 'Todos' else df[df['state'] == selected_state]
    fig = px.histogram(filtered_df, x='category', color='category', color_discrete_sequence=px.colors.sequential.Viridis)
    fig.update_layout(title="Distribución de Categorías de Transacción", paper_bgcolor='#1f2630', plot_bgcolor='#1f2630', font_color='white')
    return fig

@app.callback(
    Output('weekly-hourly-frequency', 'figure'),
    Input('state-filter', 'value')
)
def update_weekly_hourly_frequency(selected_state):
    # Filtrar por estado
    filtered_df = df if selected_state == 'Todos' else df[df['state'] == selected_state]
    
    # Crear tabla de frecuencia de transacciones por día y hora
    heatmap_data = filtered_df.groupby(['day_of_week', 'hour']).size().reset_index(name='Cantidad de Transacciones')
    
    # Ordenar días de la semana
    days_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    heatmap_data['day_of_week'] = pd.Categorical(heatmap_data['day_of_week'], categories=days_order, ordered=True)
    
    # Crear el heatmap con un degradado de un solo color (e.g., azul oscuro a azul claro)
    fig = px.imshow(
        heatmap_data.pivot('hour', 'day_of_week', 'Cantidad de Transacciones').fillna(0),
        labels=dict(x="Día de la Semana", y="Franja Horaria", color="Cantidad de Transacciones"),
        x=days_order,
        y=hour_labels,  # Etiquetas de franjas horarias detalladas
        color_continuous_scale=px.colors.sequential.Blues  # Degradado de un solo color (azul)
    )
    fig.update_layout(
        title="Interacción por Franja Horaria y Día de la Semana",
        paper_bgcolor='#1f2630', 
        font_color='white',
        height=800,  # Altura ajustada para mejor visualización de cada franja horaria
        width=1200  # Ancho ajustado para que los días se vean más amplios
    )
    return fig

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True, port=8060)